In [1]:
import os
import shutil
import pickle
import numpy as np
import torch
import copy
import matplotlib.pyplot as plt

from skopt.space import Real, Categorical, Integer

from bcnf.gp_minimize.gp_minimize import gp_minimize_fixed
from bcnf.simulation.physics import get_data
from bcnf.models.cnf import CondRealNVP
from bcnf.models.feature_network import FullyConnectedFeatureNetwork
from bcnf.eval.crossvalidate import cross_validate
from bcnf.errors import TrainingDivergedError
from bcnf.utils import get_dir

from bcnf.simulation.sampling import generate_data

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Setup

In [2]:
# HACK: scikit-optimize is not maintained anymore and this is a quick fix to make it work
# https://github.com/scikit-optimize/scikit-optimize/issues/1171#:~:text=To%20avoid%20this%20error%20in%20existing%20code%2C%20use%20int%20by%20itself.
np.int = np.int64

In [13]:
# Store and load the progress in the Huggingface repository
checkpoint_file = os.path.join(get_dir("models", "bcnf-models", "hyperparameter_optimization", "stage_2", create=True), 'checkpoint_improved.pkl')
metrics_dir = get_dir("models", "bcnf-models", "hyperparameter_optimization", "stage_2", "metrics", create=True)

print(os.path.abspath(checkpoint_file))
print(os.path.abspath(metrics_dir))

/home/psaegert/Projects/bcnf/models/bcnf-models/hyperparameter_optimization/stage_2/checkpoint_improved.pkl
/home/psaegert/Projects/bcnf/models/bcnf-models/hyperparameter_optimization/stage_2/metrics


## Data

In [5]:
param_names = ['x0_x','x0_y','x0_z','v0_x','v0_y','v0_z','g','w_x','w_y','w_z','b','A','Cd','rho','m','a_x','a_y','a_z','r']

In [4]:
# dataset_name = "hyperparameter_optimization_trajectories"

# if not os.path.exists(os.path.join(get_dir('data', 'bcnf-data'), dataset_name + '.pkl')):
#     data = generate_data(
#         n=2000,
#         type="trajectory",
#         SPF=1/30,
#         T=3,
#         config_file=os.path.join(get_dir("configs"), "throw_upwards.yaml"),
#         verbose=True,
#         break_on_impact=False,
#         name=dataset_name
#     )
# else:
#     with open(os.path.join(get_dir('data', 'bcnf-data'), dataset_name + '.pkl'), 'rb') as f:
#         data = pickle.load(f)

In [6]:
# X = np.array(data['traj'])
# y = np.column_stack([np.array(data[param]) for param in param_names])


# X_tensor = torch.Tensor(X.reshape(X.shape[0], -1))
# y_tensor = torch.Tensor(y)

# X_tensor.shape, y_tensor.shape

(torch.Size([2000, 270]), torch.Size([2000, 19]))

In [ ]:
X, y = get_data(
    T=1.0,
    dt=1 / 30,
    N=2_000,
    break_on_impact=False
)

X_tensor = torch.Tensor(X.reshape(X.shape[0], -1))
y_tensor = torch.Tensor(y)

X_tensor.shape, y_tensor.shape

## Hyperparameter

In [7]:
model_size = y_tensor.shape[1]
feature_size = X_tensor.shape[1]

print(f"Model size: {model_size}")
print(f"Feature size: {feature_size}")

Model size: 19
Feature size: 270


In [8]:
# These parameters have worked well in pilot experiments
optimizer_kwargs = {
    "lr": 2e-4
}

lr_scheduler_kwargs = {
    "mode": "min",
    "factor": 0.5,
    "patience": 250,
    "threshold_mode": "abs",
    "threshold": 1e-1,
}

In [9]:
# Define the search space
search_spaces = {
    'condition_size': Integer(1, 2048),
    'model_nested_size': Integer(16, 1024),
    'model_nested_layers': Integer(1, 8),
    'model_n_blocks': Integer(4, 32),
    'model_act_norm': Categorical([True, False]),
    'model_dropout': Real(0.0, 0.5),
    'feature_network_hidden_size': Integer(16, 256),
    'feature_network_hidden_layers': Integer(0, 16),
    'feature_network_dropout': Real(0.0, 0.5),
}

## Helper Functions

In [14]:
def param_index(name: str, search_spaces: dict[str, Real | Integer | Categorical]):
    """
    Get the index of a parameter in the search space to match the order of the parameters in the optimization function.
    
    Parameters
    ----------
    name : str
        The name of the parameter.
    search_spaces : dict[str, Real | Integer | Categorical]
        The search space.

    Returns
    -------
    int
        The index of the parameter in the search space.
    """
    return list(search_spaces.keys()).index(name)

In [15]:
def score_parameters(params: list):
    print({k: v for k, v in zip(search_spaces.keys(), params)})

    # Catch training errors (such as divergence) to filter out bad parameter sets and speed up the optimization
    try:
        # Use cross-validation to estimate the performance of the model
        fold_metrics = cross_validate(
            model_class=CondRealNVP,
            model_kwargs={
                "size": model_size,
                "nested_sizes": [params[param_index('model_nested_size', search_spaces)]] * params[param_index('model_nested_layers', search_spaces)],
                "n_blocks": params[param_index('model_n_blocks', search_spaces)],
                "n_conditions": params[param_index('condition_size', search_spaces)],
                "act_norm": params[param_index('model_act_norm', search_spaces)],
                "dropout": params[param_index('model_dropout', search_spaces)],
            },
            feature_network_class=FullyConnectedFeatureNetwork,
            feature_network_kwargs={
                "sizes": [feature_size]
                    + [params[param_index('feature_network_hidden_size', search_spaces)]] * params[param_index('feature_network_hidden_layers', search_spaces)]
                    + [params[param_index('condition_size', search_spaces)]],
                "dropout": params[param_index('feature_network_dropout', search_spaces)],    
            },
            optimizer_class=torch.optim.Adam,
            optimizer_kwargs=optimizer_kwargs,
            lr_scheduler_class=torch.optim.lr_scheduler.ReduceLROnPlateau,
            lr_scheduler_kwargs=lr_scheduler_kwargs,
            X=X_tensor,
            y=y_tensor,
            n_epochs=50_000,
            val_loss_patience=500,
            val_loss_tolerance=1e-1,  # Improvements to treat as significant
            val_loss_tolerance_mode="abs",
            timeout=60 * 60,  # 1 hour
            batch_size=256,
            device=device,
            verbose=True,  # Print the progress
            n_splits=3,
            errors="raise"  # Raise errors to stop the optimization
        )

        # Save the loss_history and metrics for analysis
        with open(os.path.join(metrics_dir, f'params_{"_".join([str(p) for p in params])}.pkl'), 'wb') as f:
            pickle.dump(fold_metrics, f)

    except TrainingDivergedError as e:
        print(e)
        return 100  # A big number (bad score) to avoid this parameter set

    # Print the average validation loss and its standard deviation during optimization
    val_loss_list = [r['val_loss'][1] for r in fold_metrics]  # each val_loss value is a tuple (epoch, loss)
    print(f'Val Loss: {np.mean(val_loss_list):.4f} ± {np.std(val_loss_list):.4f}')

    # Return the upper confidence bound of the validation loss
    # This encourages the optimization to find good AND reliable parameter sets
    return np.mean(val_loss_list) + np.std(val_loss_list)

In [16]:
def save_checkpoint(result):
    """
    Save the result of the optimization to a checkpoint file.
    Used as a callback in the optimization function.
    
    Parameters
    ----------
    result : OptimizeResult
        The result of the optimization.
    """
    # Safely write the result to a temporary file first without overwriting the checkpoint file
    with open(checkpoint_file + ".tmp", 'wb') as f:
        # Ignore
        # - result['specs']['args']['func']
        # - result['specs']['args']['callback']
        # because it causes problems when reading somewhere else
        result_no_func = copy.deepcopy(result)
        del result_no_func['specs']['args']['func']
        del result_no_func['specs']['args']['callback']
        pickle.dump(result_no_func, f)

    # Delete the old checkpoint file and rename the temporary file
    shutil.move(checkpoint_file + ".tmp", checkpoint_file)

## Optimization

In [17]:
# Numer of random initial points to explore the search space
N_STEPS_INIT = 30

# Number of iterations to run the optimization in total
N_STEPS = 100

In [18]:
# Load the checkpoint if it exists
if os.path.exists(checkpoint_file):
    print(f'Loading checkpoint from {checkpoint_file}')

    with open(checkpoint_file, 'rb') as f:
        checkpoint = pickle.load(f)

        # Re-assign the function and callback because they are not picklable
        checkpoint['specs']['args']['func'] = score_parameters
        checkpoint['specs']['args']['callback'] = save_checkpoint

    print(f'Resuming from iteration {len(checkpoint.x_iters)}')

    n_initial_points = max(0, N_STEPS_INIT - len(checkpoint.x_iters))
    n_calls_remaining = max(0, N_STEPS - len(checkpoint.x_iters))
    x0 = checkpoint.x_iters
    y0 = checkpoint.func_vals
else:
    print('No checkpoint found. Starting new optimization')
    checkpoint = None

    n_initial_points = N_STEPS_INIT
    n_calls_remaining = N_STEPS

    x0 = None
    y0 = None
    
print(f'Running with {n_initial_points} initial points and {n_calls_remaining} remaining iterations')

No checkpoint found. Starting new optimization
Running with 30 initial points and 100 remaining iterations


In [19]:
# You might want to adjust the n_calls or other parameters based on the checkpoint
result = gp_minimize_fixed(
    func=score_parameters,
    dimensions=search_spaces.values(),
    n_initial_points=n_initial_points,  # Number of random points before starting the optimization
    n_calls=n_calls_remaining,  # Number of iterations
    random_state=2024_03_25,
    verbose=True,
    acq_func="EI",  # Expected Improvement https://arxiv.org/pdf/1009.5419.pdf
    initial_point_generator="halton", # https://en.wikipedia.org/wiki/Halton_sequence
    callback=save_checkpoint,
    x0=x0,
    y0=y0)

30 initial points will be randomly generated
Iteration No: 1 started. Evaluating function at random point.
{'condition_size': 1, 'model_nested_size': 16, 'model_nested_layers': 1, 'model_n_blocks': 4, 'model_act_norm': False, 'model_dropout': 0.0, 'feature_network_hidden_size': 16, 'feature_network_hidden_layers': 0, 'feature_network_dropout': 0.0}


Train: 24.6184 - Val: 36.4657 (avg: 36.3351, min: 31.8462) | lr: 5.00e-05 - Patience: 500/500:   4%|▎         | 1812/50000 [00:41<18:31, 43.34it/s]     
Train: 24.2726 - Val: 26.2805 (avg: 26.2549, min: 26.0513) | lr: 1.00e-04 - Patience: 500/500:   3%|▎         | 1435/50000 [00:32<18:32, 43.67it/s]    
Train: 21.4233 - Val: 25.5778 (avg: 25.4570, min: 23.9413) | lr: 1.00e-04 - Patience: 500/500:   5%|▌         | 2551/50000 [00:58<18:14, 43.36it/s]    


Val Loss: 26.6614 ± 1.5437
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 133.9515
Function value obtained: 28.2051
Current minimum: 28.2051
Iteration No: 2 started. Evaluating function at random point.
{'condition_size': 1024, 'model_nested_size': 352, 'model_nested_layers': 2, 'model_n_blocks': 8, 'model_act_norm': False, 'model_dropout': 0.038461538461538464, 'feature_network_hidden_size': 30, 'feature_network_hidden_layers': 1, 'feature_network_dropout': 0.021739130434782608}


Train: -0.4672 - Val: 50.7001 (avg: 49.7093, min: 21.1193) | lr: 1.00e-04 - Patience: 500/500:   1%|          | 608/50000 [00:26<35:19, 23.30it/s]
Train: -0.2403 - Val: 49.4795 (avg: 48.9554, min: 20.5015) | lr: 1.00e-04 - Patience: 500/500:   1%|          | 621/50000 [00:26<35:04, 23.46it/s]
Train: -1.0247 - Val: 46.1674 (avg: 45.8817, min: 19.3636) | lr: 1.00e-04 - Patience: 500/500:   1%|          | 623/50000 [00:26<35:23, 23.25it/s]


Val Loss: 50.4280 ± 1.2651
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 79.4270
Function value obtained: 51.6931
Current minimum: 28.2051
Iteration No: 3 started. Evaluating function at random point.
{'condition_size': 513, 'model_nested_size': 688, 'model_nested_layers': 4, 'model_n_blocks': 12, 'model_act_norm': False, 'model_dropout': 0.07692307692307693, 'feature_network_hidden_size': 44, 'feature_network_hidden_layers': 2, 'feature_network_dropout': 0.043478260869565216}


...

Val Loss: 85.4297 ± 5.5201
Iteration No: 18 ended. Evaluation done at random point.
Time taken: 341.9426
Function value obtained: 90.9498
Current minimum: 14.8708
Iteration No: 19 started. Evaluating function at random point.
{'condition_size': 577, 'model_nested_size': 91, 'model_nested_layers': 6, 'model_n_blocks': 21, 'model_act_norm': True, 'model_dropout': 0.1952662721893491, 'feature_network_hidden_size': 31, 'feature_network_hidden_layers': 15, 'feature_network_dropout': 0.391304347826087}


Train: 13.2733 - Val: 19.1405 (avg: 19.0148, min: 18.6063) | lr: 1.00e-04 - Patience: 500/500:   3%|▎         | 1605/50000 [04:49<2:25:28,  5.54it/s]
Train: 13.8483 - Val: 17.4437 (avg: 17.5042, min: 17.3523) | lr: 1.00e-04 - Patience: 500/500:   3%|▎         | 1616/50000 [05:09<2:34:12,  5.23it/s]
Train: 12.7858 - Val: 17.3372 (avg: 17.4055, min: 17.2604) | lr: 1.00e-04 - Patience: 500/500:   4%|▍         | 1882/50000 [05:45<2:27:14,  5.45it/s]


Val Loss: 17.8375 ± 0.3759
Iteration No: 19 ended. Evaluation done at random point.
Time taken: 944.1521
Function value obtained: 18.2135
Current minimum: 14.8708
Iteration No: 20 started. Evaluating function at random point.
{'condition_size': 1600, 'model_nested_size': 427, 'model_nested_layers': 7, 'model_n_blocks': 25, 'model_act_norm': True, 'model_dropout': 0.23372781065088757, 'feature_network_hidden_size': 45, 'feature_network_hidden_layers': 0, 'feature_network_dropout': 0.41304347826086957}


Train: -6.8219 - Val: 62.2505 (avg: 61.8562, min: 13.8937) | lr: 1.00e-04 - Patience: 500/500:   1%|▏         | 646/50000 [02:36<3:19:11,  4.13it/s]
Train: -7.1767 - Val: 55.9786 (avg: 53.0200, min: 12.6566) | lr: 1.00e-04 - Patience: 500/500:   1%|▏         | 643/50000 [02:41<3:26:04,  3.99it/s]
Train: -0.2989 - Val: 29.7592 (avg: 26.4453, min: 13.4581) | lr: 2.00e-04 - Patience: 210/500:   1%|          | 346/50000 [01:32<3:40:11,  3.76it/s]


KeyboardInterrupt: 